In [1]:
import sys

sys.path.append(f"/home/npanj/personal_works/m5-forecasting-accuracy")

from src.utils.import_downcasting import import_downcasting
from src.utils.plotting import PdfFile
from utils import get_wmape, plotting_sales_forecast, get_wmape_custom_groupby
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import copy

root = os.path.join(
    "/home/npanj/personal_works/m5-forecasting-accuracy/assets/data/data_CA_1"
)

In [23]:
data_ca1 = import_downcasting(os.path.join(root, "data_with_arima_resid.csv"))
score_ca1 = import_downcasting(os.path.join(root, "arima_model_score.csv"))

print("data ca1 list columns")
print(data_ca1.info(verbose=True))

print("--------------------------------------------------")
print("data ca1 score list columns")
print(score_ca1.info())

data_ca1 = data_ca1.set_index("date")
data_ca1["arima_residual"] = data_ca1.arima_residual.astype(np.float64)
data_ca1["event_name_1"] = (
    data_ca1["event_name_1"].cat.add_categories("none").fillna("none")
)
data_ca1["event_type_1"] = (
    data_ca1["event_type_1"].cat.add_categories("none").fillna("none")
)
data_ca1["event_name_2"] = (
    data_ca1["event_name_2"].cat.add_categories("none").fillna("none")
)
data_ca1["event_type_2"] = (
    data_ca1["event_type_2"].cat.add_categories("none").fillna("none")
)

data_ca1["event_name_1_group"] = data_ca1["event_name_1"].apply(lambda x: x if x == "none" else "special" )
data_ca1["event_type_1_group"] = data_ca1["event_type_1"].apply(lambda x: x if x == "none" else "special" )
data_ca1.info()

data ca1 list columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918109 entries, 0 to 5918108
Data columns (total 25 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   id              category      
 2   item_id         category      
 3   dept_id         category      
 4   cat_id          category      
 5   store_id        category      
 6   state_id        category      
 7   date_code       category      
 8   sales           int16         
 9   wm_yr_wk        int16         
 10  weekday         category      
 11  wday            int8          
 12  month           int8          
 13  year            int16         
 14  d               category      
 15  event_name_1    category      
 16  event_type_1    category      
 17  event_name_2    category      
 18  event_type_2    category      
 19  snap_CA         int8          
 20  snap_TX         int8          
 21  snap_WI         int8          
 

- event
  May need to combine between event_1 and event_2

- snap
  Focus only on snap_CA (CA = Califonia, TX = texas, WI = witcostin)

In [24]:
##### Checking event_name_1 and event_type_1
# check_event_name_type_1 = data_ca1[["event_name_1", "event_type_1", "sales"]].groupby(["event_name_1", "event_type_1"]).count()\
# .reset_index()
# check_event_name_type_1 = check_event_name_type_1[check_event_name_type_1.sales > 0]
# check_event_name_type_1.sort_values(by="event_type_1", inplace=True)
# check_event_name_type_1


##### Checking event_name_2 and event_type_2
data_ca1[["event_name_2", "sales"]].groupby("event_name_2").count()

##### Checking snap_CA
# check_snap = data_ca1[["snap_CA", "sales"]].groupby(["snap_CA"]).count()\
# .reset_index()
# check_snap.rename(columns={"sales": "num_obs"}, inplace=True)
# check_snap

##### Checking dept_id and cat_id
# check_dept_cat = data_ca1[["dept_id", "cat_id", "sales"]].groupby(["dept_id", "cat_id"]).count()\
# .reset_index()
# check_dept_cat.rename(columns={"sales": "num_obs"}, inplace=True)
# check_dept_cat = check_dept_cat[check_dept_cat.num_obs > 0]
# check_dept_cat

,sales
event_name_2,
Cinco De Mayo,3049
Easter,3049
Father's day,3049
OrthodoxEaster,3049
none,5905913


In [4]:
wmape = get_wmape(data_ca1, "arima_residual")
score_ca1 = pd.merge(
    left=score_ca1,
    right=wmape[["store_id", "item_id", "wmape"]],
    on=["store_id", "item_id"],
    how="left",
)

score_ca1.sort_values(by="wmape", ascending=False, inplace=True)
score_ca1["rank_pct_wmape"] = score_ca1["wmape"].rank(pct=True)
score_ca1 = score_ca1.reset_index(drop=True)

data_ca1["root_square_resid"] = data_ca1["arima_residual"].apply(
    lambda x: math.pow(x, 2)
)

In [5]:
item_id_cat_unq = data_ca1.groupby(["item_id", "cat_id"]).store_id.count().reset_index()
score_ca1 = pd.merge(
    left=score_ca1,
    right=item_id_cat_unq[["item_id", "cat_id"]],
    left_on=["item_id"],
    right_on=["item_id"],
    how="left",
)

score_ca1.info()
##########

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9147 entries, 0 to 9146
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   store_id        9147 non-null   category
 1   item_id         9147 non-null   category
 2   mse             9147 non-null   float16 
 3   mae             9147 non-null   float16 
 4   sse             9147 non-null   float32 
 5   wmape           9147 non-null   float64 
 6   rank_pct_wmape  9147 non-null   float64 
 7   cat_id          9147 non-null   category
dtypes: category(3), float16(2), float32(1), float64(2)
memory usage: 410.2 KB


In [6]:
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_store_ca1.pdf")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_hobbies_ca1.pdf", product_cat="HOBBIES")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_foods_ca1.pdf", product_cat="FOODS")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_household_ca1.pdf",\
#                         product_cat="HOUSEHOLD")

# Analysis WMAPE on event_name_type1

- event_name_1 = "Christmas" has sales equal to 0 for all observastion
- Marjoriry of data belong to None type
- Only these following days that seem to have impact that cuase residual to be higher
        Thanksgiving
        NewYear
        Halloween
        LentStart
        LentWeek2
        ValentinesDay
        NBAFinalsStart
- grouping into two group comsprised of special and none. These feature seem to have little of impact 

In [7]:
wmape_by_event_name_1 = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_name_1"])
wmape_by_event_name_1.to_csv("wmape_analysis/wmape_by_event_name_1.csv")

wmape_event_name_1_grp = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_name_1_group"])
wmape_event_name_1_grp.to_csv("wmape_analysis/wmape_event_name_1_group.csv")

# Analysis WMAPE on event_type_1

- Only these following days that seem to have impact that cuase residual to be higher
        National (Highest Different)
        Cultural
- Grouping to special and non special seem to have none impact

In [8]:
wmape_by_event_type_1 = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_type_1"])
wmape_by_event_type_1.to_csv("wmape_analysis/wmape_by_event_type_1.csv")

wmape_by_event_type_1_group = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_type_1_group"])
wmape_by_event_type_1_group.to_csv("wmape_analysis/wmape_by_event_type_1_group.csv")

# Analysis WMAPE on snap_CA

- WMAPE seems indifferent between having a snap and not having a snap

In [9]:
wmape_snap = get_wmape_custom_groupby(data_ca1, "arima_residual", ["snap_CA"])
wmape_snap.to_csv("wmape_analysis/wmape_snap.csv")

# Analysis WMAPE on dept_id

Department ID has potential to adding information to the model

In [10]:
wmape_dept = get_wmape_custom_groupby(data_ca1, "arima_residual", ["dept_id"])
wmape_dept.to_csv("wmape_analysis/wmape_dept.csv")

wmape_dept

,dept_id,sum_abs_residual,mean_abs_residual,num_obs,sales,avg_sales,wmape,pct_obs
4,HOBBIES_2,8.409791e+04,0.290786,289209,56505.0,0.195378,1.488327,0.048868
6,HOUSEHOLD_2,4.114642e+05,0.411623,999615,347116.0,0.347250,1.185380,0.168908
3,HOBBIES_1,7.721161e+05,0.956233,807456,835578.0,1.034828,0.924050,0.136438
1,FOODS_2,6.736336e+05,0.871997,772518,900391.0,1.165528,0.748157,0.130535
0,FOODS_1,4.282969e+05,1.021564,419256,577436.0,1.377287,0.741722,0.070843
5,HOUSEHOLD_1,8.188265e+05,0.792966,1032612,1121388.0,1.085972,0.730190,0.174483
2,FOODS_3,2.248673e+06,1.407670,1597443,3993834.0,2.500142,0.563036,0.269925


# Analysis WMAPE on cat_id

In [16]:
wmape_cat = get_wmape_custom_groupby(data_ca1, "arima_residual", ["cat_id"])
wmape_cat.to_csv("wmape_analysis/wmape_cat.csv")
wmape_cat

,cat_id,sum_abs_residual,mean_abs_residual,num_obs,sales,avg_sales,wmape,pct_obs
1,HOBBIES,8.562141e+05,0.780743,1096665,892083.0,0.813451,0.959792,0.185307
2,HOUSEHOLD,1.230291e+06,0.605390,2032227,1468504.0,0.722608,0.837785,0.343391
0,FOODS,3.350604e+06,1.201270,2789217,5471661.0,1.961719,0.612356,0.471302


# Analysis residual from ARIMA model in the group last 20 percentile

In [12]:
resid_score_worst_20pct = score_ca1[score_ca1.rank_pct_wmape > 0.8]
item_resid_worst_20pct = list(resid_score_worst_20pct.item_id.unique())

data_worst_20pct = data_ca1[data_ca1.item_id.isin(item_resid_worst_20pct)]
data_worst_20pct.to_csv(os.path.join(root, "data_arima_worst_30pct.csv"))

data_worst_20pct = import_downcasting(os.path.join(root, "data_arima_worst_30pct.csv"))